### Load preprocessed data

If you'd like to play around with this notebook, start by downloading the skipgram dataset from here:

https://www.dropbox.com/s/nd1zxh538o6psal/skipgram_full.npz

WARNING: it's a 1Gb download, so it may take a while!

In [1]:
# !wget  -q https://www.dropbox.com/s/nd1zxh538o6psal/skipgram_full.npz

In [2]:
import numpy as np

codes = np.load("skipgram_full.npz")['coded']
# Remove duplicate skipgrams
codes = codes[codes[:, 0] != codes[:, 1]]
code2token = np.load("skipgram_full.npz", allow_pickle=True)['c2t'].tolist()
token2code = np.load("skipgram_full.npz", allow_pickle=True)['t2c'].tolist()

# First column is the first token code
# second column is the 2nd token code
# third column is the skip gram count
# fourth is PMI * 1e6
codes

array([[  13835,    3257,    4605,  592814],
       [  12071,    3257,      16,  491071],
       [   4136,    3257,       2, -621270],
       ...,
       [  12293,    1390,       1, 1092727],
       [   5103,    1390,       1, 2368132],
       [   6789,    1390,       1,  427689]], dtype=int32)

In [3]:
train_x = codes[:, :2].copy().astype(np.int64)
train_y = codes[:, 3].astype(np.float32) / 1e6
train_y, train_y.max()

(array([ 0.592814,  0.491071, -0.62127 , ...,  1.092727,  2.368132,
         0.427689], dtype=float32),
 12.09618)

In [4]:
top_codes = np.argsort(train_y)[-10:]
[[code2token[c[0]], code2token[c[1]]] for c in codes[top_codes, :2]]

[['norris', 'roundhouse'],
 ['palpatine', 'skywalker'],
 ['palpatine', 'sith'],
 ['roundhouse', 'norris'],
 ['lankan', 'sri'],
 ['palpatine', 'anakin'],
 ['skywalker', 'palpatine'],
 ['anakin', 'palpatine'],
 ['blahblah', 'blah'],
 ['blah', 'blahblah']]

In [5]:
n_user = np.max(train_x[:, :2]) + 1
n_item = np.max(train_x[:, :2]) + 1
n_user

14003

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from abstract_model import AbstractModel

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(AbstractModel):
    def __init__(self, n_user, n_item, train_x, train_y, test_x, test_y, 
                 k=128, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        self.save_data(train_x, train_y, test_x, test_y)
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))

    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # Add bias prediction to the interaction prediction
        prediction = ui_interaction + biases
        return prediction
    
    def likelihood(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse
    
    def prior(self):
        # Add new regularization to the biases
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = prior_user + prior_item + prior_bias_user + prior_bias_item
        return total


#  For an unsupervised model, we're not  interested
# in  the holdout metrics, so there is no  test set
model = MF(n_user, n_item, train_x, train_y, train_x, train_y)

# add a logger
logger = TensorBoardLogger("tb_logs", name="w2v_model")

# We could have turned on multiple GPUs here, for example
# trainer = pl.Trainer(gpus=8, precision=16)    
trainer = pl.Trainer(max_epochs=5,
                     reload_dataloaders_every_epoch=True,
                     logger=logger)    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


#### Run model

In [ ]:
trainer.fit(model)


  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 1 M   
1 | item      | Embedding | 1 M   
2 | bias_user | Embedding | 14 K  
3 | bias_item | Embedding | 14 K  


In [ ]:
torch.save(model.state_dict(), "model_05_word2vec")

### Introspect the model

In [ ]:
vectors_raw = model.user.weight.data.numpy()
vectors = vectors_raw / np.sqrt((vectors_raw**2.0).sum(axis=1)[:, None])

In [ ]:
(vectors[0]**2.0).sum()

In [ ]:
def find_closest(token, n=10):
    code = token2code[token]
    vector = vectors[code]
    similarity = np.sum(vector[None, :] * vectors, axis=1)
    closest = np.argsort(similarity)[::-1]
    for code in closest[1:n]:
        print(code2token[code], similarity[code])

In [ ]:
find_closest('lol')

In [ ]:
find_closest('hipster')

In [ ]:
find_closest('groovy')

In [ ]:
find_closest('bromance')

In [ ]:
find_closest('barbie')

In [ ]:
find_closest('relationship')

In [ ]:
find_closest('pope')

In [ ]:
find_closest('selfie')

### Subtract and add word vectors

In [ ]:
def add_subtract(center, minus, plus, n=10):
    vector = (vectors[token2code[center]]
             - vectors[token2code[minus]]
             + vectors[token2code[plus]])
    similarity = np.sum(vector[None, :] * vectors, axis=1)
    closest = np.argsort(similarity)[::-1]
    for code in closest[2:n]:
        tok = code2token[code]
        if tok != center and tok != minus and tok != plus:
            print(code2token[code])

In [ ]:
add_subtract('burrito', 'mexican', 'italian')